In [1]:
import numpy as np
import os
import pandas as pd

In [2]:
p_in = './../data/in/gene_info_downloaded_170216.gz' # manually downloaded
p_out = './../data/internal/batches_for_genealacart'
os.makedirs(p_out)

In [3]:
df = pd.read_table(
    p_in,
    sep='\t',
    header=0)

In [4]:
# presenly GeneALaCart only supports human genes
t = 9606 

f = (df['type_of_gene'] == 'protein-coding') &\
    (df['#tax_id'] == t)
    
# GeneALaCart appears to use symbol_authority and not symbol_ncbi 
# Tested the mapping of a mitochondrial gene(mapping to MT-ND5 rather than ND5)
# Update: 16-10-20: This is not entirely true, as there also is some ambiguity
# in their internal mapping -> see /docs/genealacart.txt
d = df.loc[f, 'Symbol_from_nomenclature_authority']

# exclude genes without proper symbol (seem to be genes having only locus name in ncbi )
f = d == '-' 
d = d[~f]

In [5]:
# create batche

total_row = d.shape[0]
batch_size=2000
ixs = np.arange(1, total_row,batch_size)
ixs = np.append(ixs, total_row+1)

ixs[0] = 0

for j in range(len(ixs)-1):
    s = ixs[j]
    e = ixs[j+1]
    
    dc = d[s:e]
        
    p = os.path.join(
        p_out,    
        'symbol_authority_protein_coding_batch_' + str(j) + '.txt')
    dc.to_csv(p, index=False)